### Use the first cell to load libraries, set up the census API, and download variable names for detailed tables, summary tables, and data profile tables

In [384]:
import pandas as pd
from census import Census
import json
import urllib
import re

pd.set_option('max_columns',100)
c = Census('e3e84acee03bed32ab1303ef49240f9481298969')

details_url = 'https://api.census.gov/data/2015/acs1/variables.json'
subject_url = 'https://api.census.gov/data/2015/acs1/subject/variables.json'
data_profile_url = 'https://api.census.gov/data/2015/acs1/profile/variables.json'

def get_census_vars(url):
    '''Download and format census variables'''
    
    vars_url = urllib.urlopen(url)
    vars_url = json.load(vars_url)
    
    df = pd.DataFrame(vars_url['variables']).T
    
    # Filter down to rows with acutal variables and only concept and desciption columns
    df = df.loc[df.index.str.contains('^B[0-9]{5}|^S[0-9]{3}|^DP[0-9]{2}'),['concept','label']]
    return df

details = get_census_vars(details_url)
subject = get_census_vars(subject_url)
data_profile = get_census_vars(data_profile_url)

In [385]:
details = details.loc[details.index.str.contains('^B01001_...E')]

#Build fields tuple
fields = tuple(['NAME'] + list(details.index))

In [435]:
def get_data(fields, time_type):
    '''
    Retrieves data from the Census API, 
    grabbing subidivision data if time_type is 5, 
    and county data if time_type is 1
    
    Return: Dataframe
    '''
    
    if time_type == 5:
        
        data = c.acs5.get(fields,geo={'for':'county subdivision:*',
                               'in':'state:25 county:11,13,15'},
                                year='2015')
    elif time_type == 1:
        
        data = c.acs1.get(fields,geo={'for':'county:11,13,15',
                               'in':'state:25'},
                                year='2015')
        
    else: raise Exception('Invalid time type')
        
    data = pd.DataFrame(data)
    return data

# Prettify geography names
def prettify_geo_names(col,time_type):
    if time_type == 5:
        col = col.apply(lambda x : re.match('(.+?)(?=\s[Tt]own|\s[Cc]ity)', x).group())
    elif time_type == 1:
        col = col.apply(lambda x : re.match('(.+?)County', x).group())
    return col

def add_columns_by_name(col,df):
    '''
    For adding male and female population columns
    '''
    
    columns_to_add = df.columns.str.contains(col)
    summed = df.loc[:,columns_to_add].sum(axis=1)
    return summed  

In [436]:
def build_ages_table(fields,time_type):

    ages = get_data(fields, time_type)

    # Generate intelligible column names from the variables index
    new_columns = [details.loc[x,'label'] for x in ages.columns if x in details.index]

    # Replace variable column names
    ages.columns = new_columns + list(ages.columns[len(new_columns):])

    ages['NAME'] = prettify_geo_names(ages['NAME'],time_type)    
    ages.index = ages['NAME']
    ages = ages.apply(lambda x : x.astype('float64',errors='ignore'))
    ages
    
    #Write raw data to a csv file
    ages.to_csv('{}_year_raw_data'.format(time_type))

    # Select all alphanumeric chars and spaces after the exclamation points
    regex = '[\w\s]+$'

    # Create column names for totals dataframe
    # TODO - clean up how I create this columns
    total_cols = pd.Series([re.search(regex,col).group() for col in new_columns if '!!' in col]).unique()

    # Create a totals dataframe with the index from the ages dataframe
    totals = pd.DataFrame(columns=total_cols, index=ages.index)

    # Promote columns to a row so that we can access whie apply a lmbada to the dataframe
    # This row will disappear while applying add_columns_by_name
    totals.loc['_cols'] = totals.columns

    totals = totals.apply(lambda x : add_columns_by_name(x.loc['_cols'], ages))
    return totals

In [437]:
subdivisions = build_ages_table(fields,5)
counties = build_ages_table(fields,1)

counties.loc['Pioneer Valley'] = counties.loc[['Franklin County','Hampden County','Hampshire County']].sum()
counties.loc['Pioneer Valley Region'] = counties.loc[['Hampden County','Hampshire County']].sum()

ages_table = pd.concat([counties,subdivisions])

In [439]:
ages_table.to_csv('ages_tables.csv')